In [1]:
from lightfm import LightFM
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

/Users/alextsai/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
%%time
listen_data = pd.read_table('data/train_triplets.txt', header = None)
listen_data.columns = ['user_id','item_id','listen_count']

CPU times: user 31.7 s, sys: 3.31 s, total: 35.1 s
Wall time: 35.4 s


In [3]:
small_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
listen_data_small = pd.read_table(small_file, header = None)
listen_data_small.columns = ['user_id','item_id','listen_count']

In [4]:
value_matrix = listen_data_small.pivot(index = 'user_id',columns='item_id',values='listen_count')
value_matrix = value_matrix.fillna(0)
value_matrix = csr_matrix(value_matrix)

In [19]:
song_data = pd.read_csv('data/song_data (1).csv')
song_data

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0
...,...,...,...,...,...
999995,SOTXAME12AB018F136,O Samba Da Vida,Pacha V.I.P.,Kiko Navarro,0
999996,SOXQYIQ12A8C137FBB,Jago Chhadeo,Naale Baba Lassi Pee Gya,Kuldeep Manak,0
999997,SOHODZI12A8C137BB3,Novemba,Dub_Connected: electronic music,Gabriel Le Mar,0
999998,SOLXGOR12A81C21EB7,Faraday,The Trance Collection Vol. 2,Elude,0


In [6]:
model_fm = LightFM()
model_fm.fit(value_matrix)

In [24]:
song_df = pd.merge(listen_data_small, song_data.drop_duplicates(['song_id']), left_on="item_id", right_on="song_id",how="left")

In [30]:
users = song_df['user_id'].unique()
songs = song_df['title'].unique()

In [32]:
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage']  = song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.sort_values(['listen_count', 'song'], ascending = [0,1])


KeyError: 'song'